In [ ]:
es_index = "./es-indices.txt"
disk_size = 3000

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt, rcParams

pd.set_option('display.width', 1000)
plt.style.use('dark_background')

rcParams['font.family'] = 'VL Gothic'
rcParams['font.weight'] = 'regular'
rcParams['legend.fontsize'] = 20
rcParams['axes.titlesize'] = 20
rcParams['xtick.labelsize'] = 20
rcParams['ytick.labelsize'] = 20


In [ ]:
import re
def normalize(size):
    size, *unit = re.findall(r'([0-9.]+|\D+)',size)
    if (unit == 'kb'):
       size = float(size) * 1024
    elif (unit == 'mb'):
       size = float(size) * 1024 * 1024
    elif (unit == 'gb'):
       size = float(size) * 1024 * 1024 * 1024
    return int(float(size)) #/ 1024 / 1024


In [ ]:
df = pd.read_csv(es_index, delim_whitespace=True)

sizedf = pd.concat([df[['index', 'store.size']],df['index'].str.extract('(?P<index_series>.*)-(?P<date>\d{4}.\d{2}.\d{2})', expand=True)], axis=1)
sizedf['store.size'] = sizedf['store.size'].map(normalize)
sizedf['date'] = pd.to_datetime(sizedf['date'])
sizedf.dropna(inplace=True)

index_df = pd.DataFrame(sizedf.drop_duplicates('date').sort_values(by=['date'])['date'].reset_index(drop=True))
for lindex in sizedf['index_series'].drop_duplicates():
    index_df = pd.merge(index_df, sizedf[sizedf.index_series == lindex][['date','store.size']].rename(columns={'store.size':lindex}), on='date', how='outer')
index_df.set_index('date', inplace=True)
index_df.fillna(0, inplace=True)

In [ ]:
ax = index_df.plot.bar(figsize=(20,8), stacked=True)
ax.set_title('インデックスサイズ(日毎)')
plt.show()

In [ ]:
ax = index_df.cumsum().plot.bar(figsize=(20,8), stacked=True)
ax.plot([-1,len(index_df.index)],[disk_size, disk_size], label='最大値', color='red')
ax.legend()
ax.set_title('インデックスサイズ累計値')
plt.show()